# Домашнее задание «Продвинутый pandas 2»

## Преподаватель: Константин Башевой

## Студент: Григорий Сотский

### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
ratings_grouped = ratings.groupby(ratings.userId).count()[['rating']]
ratings_grouped.head()

,rating
userId,
1,20
2,76
3,51
4,204
5,100


In [4]:
ratings_filtered = ratings_grouped[ratings_grouped.rating >= 100]

In [6]:
ratings_filtered.rename(columns={'rating': 'cnt'}, inplace=True)

In [7]:
ratings_filtered.head()

,cnt
userId,
4,204
5,100
8,116
15,1700
17,363


In [8]:
ratings_timestamp_max = ratings.groupby(ratings.userId).max()[['timestamp']]
ratings_timestamp_min = ratings.groupby(ratings.userId).min()[['timestamp']]

In [9]:
ratings_timestamp_max.rename(columns={'timestamp': 'timestamp_max'}, inplace=True)

In [10]:
ratings_timestamp_min.rename(columns={'timestamp': 'timestamp_min'}, inplace=True)

In [11]:
ratings_timestamp_max.head()

,timestamp_max
userId,
1,1260759205
2,835356246
3,1298932787
4,949982274
5,1163375145


In [12]:
ratings_timestamp_min.head()

,timestamp_min
userId,
1,1260759108
2,835355395
3,1298861589
4,949778714
5,1163373044


In [13]:
ratings_timestamp = ratings_timestamp_max.join(ratings_timestamp_min, how='left')
ratings_timestamp.head()

,timestamp_max,timestamp_min
userId,,
1,1260759205,1260759108
2,835356246,835355395
3,1298932787,1298861589
4,949982274,949778714
5,1163375145,1163373044


In [14]:
ratings_timestamp['timestamp_max'] = pd.to_datetime(ratings_timestamp['timestamp_max'], unit='s')
ratings_timestamp['timestamp_min'] = pd.to_datetime(ratings_timestamp['timestamp_min'], unit='s')

In [15]:
ratings_timestamp.head()

,timestamp_max,timestamp_min
userId,,
1,2009-12-14 02:53:25,2009-12-14 02:51:48
2,1996-06-21 11:24:06,1996-06-21 11:09:55
3,2011-02-28 22:39:47,2011-02-28 02:53:09
4,2000-02-08 03:57:54,2000-02-05 19:25:14
5,2006-11-12 23:45:45,2006-11-12 23:10:44


In [16]:
def ts(row):
    for timeline in row:
        timeline = row.timestamp_max - row.timestamp_min
    return timeline

In [17]:
ratings_timestamp['timeline'] = ratings_timestamp.apply(ts, axis=1)

In [18]:
ratings_timestamp.head()

,timestamp_max,timestamp_min,timeline
userId,,,
1,2009-12-14 02:53:25,2009-12-14 02:51:48,0 days 00:01:37
2,1996-06-21 11:24:06,1996-06-21 11:09:55,0 days 00:14:11
3,2011-02-28 22:39:47,2011-02-28 02:53:09,0 days 19:46:38
4,2000-02-08 03:57:54,2000-02-05 19:25:14,2 days 08:32:40
5,2006-11-12 23:45:45,2006-11-12 23:10:44,0 days 00:35:01


In [19]:
ratings_timestamp_ = ratings_timestamp.loc[:, ['timeline']]

In [20]:
ratings_timestamp_.head()

,timeline
userId,
1,0 days 00:01:37
2,0 days 00:14:11
3,0 days 19:46:38
4,2 days 08:32:40
5,0 days 00:35:01


In [21]:
ratings_timeline = ratings_filtered.join(ratings_timestamp_, how='left')
ratings_timeline.head()

,cnt,timeline
userId,,
4,204,2 days 08:32:40
5,100,0 days 00:35:01
8,116,0 days 23:39:47
15,1700,5455 days 22:38:16
17,363,0 days 02:14:13


### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [22]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [23]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [24]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [25]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [26]:
client_id = client_base.groupby('client_id').count()

client_id

,address
client_id,
111,1
112,1
113,1
114,1
115,1
116,1
117,1
118,1


In [27]:
rzd_ = rzd.groupby('client_id').sum()[['rzd_revenue']]
air_ = air.groupby('client_id').sum()[['air_revenue']]
auto_ = auto.groupby('client_id').sum()[['auto_revenue']]

In [28]:
revenue = client_id.join(rzd_, how='left')
revenue = revenue.loc[:, ['rzd_revenue']]
revenue = revenue.join(air_, how='left')
revenue = revenue.join(auto_, how='left')
revenue

,rzd_revenue,air_revenue,auto_revenue
client_id,,,
111,1093.0,NaN,NaN
112,2810.0,NaN,NaN
113,10283.0,NaN,57483.0
114,5774.0,NaN,83.0
115,981.0,81.0,912.0
116,NaN,4.0,4834.0
117,NaN,13.0,98.0
118,NaN,173.0,NaN


In [29]:
revenue.reset_index()
revenue_ = revenue.merge(client_base, how='left', on='client_id')
revenue_

,client_id,rzd_revenue,air_revenue,auto_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,NaN,57483.0,Левобережная 1а
3,114,5774.0,NaN,83.0,Мира 14
4,115,981.0,81.0,912.0,ЗЖБИиДК 1
5,116,NaN,4.0,4834.0,Строителей 18
6,117,NaN,13.0,98.0,Панфиловская 33
7,118,NaN,173.0,NaN,Мастеркова 4


### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1. Скорее всего координаты пользователя появились из визитов и покупок, 
поэтому какой смысл их обратно перетягивать из базы пользователей, тем более, что однозначного соответствия уже не будет?!
Мне кажется это бессмысленным действием.
2. Пользу от координат я вижу, если например с помощью координат вычислить по категориям покупок наибольшую плотность к привязке к картам, и тогда в этих местах можно будет организовать магазины, шоурумы или пункты доставки. Исходя из такой постановки, я бы сделал merge к таблице с покупками таблицу с координатами, и потом сделал сводную таблицу, как внизу в примере. Исходя из той логики, что если у 1-го пользователя только 1 покупка, но у него есть 2 точки координат, то для этой категории покупок не важно из какой точно координаты данный пользователь делал покупку. Если он бывает в 2-х точках, то товар ему может быть предложен в любой из его месторасположений. (хотя, конечно, эта гипотеза может и не сработать, если он товары для дачи покупает только на даче. Но в этом случае см. п.1, и координаты нужно брать из покупок)
3. С визитами аналогичная логика.


In [30]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99, 22, 55, 77],
        'category': ['Электроника', 'Авто', 'Дача', 'Спорт', 'Авто', 'Электроника', 'Электроника', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Электроника
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто
5,22,Электроника
6,55,Электроника
7,77,Авто


In [31]:
coordinate = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77, 99, 55],
        'lat:long': ['55.834784:37.565415', '53.890790:27.525773', '55.733670:37.587874', '55.824108:37.550309', '55.846231:37.532002', '56.011552:36.545722', '55.812308:27.525555'],
    }
)

coordinate = coordinate[['user_id', 'lat:long']]
coordinate

,user_id,lat:long
0,11,55.834784:37.565415
1,22,53.890790:27.525773
2,55,55.733670:37.587874
3,11,55.824108:37.550309
4,77,55.846231:37.532002
5,99,56.011552:36.545722
6,55,55.812308:27.525555


lat и long - это конечно 2 столбца, с которыми можно работать подключив api с картам. Для демонстрации идеи, объединил их в 1 поле

In [32]:
joined = purchases.merge(coordinate, how='left', on ='user_id')
joined.head()

,user_id,category,lat:long
0,11,Электроника,55.834784:37.565415
1,11,Электроника,55.824108:37.550309
2,22,Авто,53.890790:27.525773
3,55,Дача,55.733670:37.587874
4,55,Дача,55.812308:27.525555


In [33]:
joined.pivot_table(index = 'lat:long', columns = 'category', values = 'user_id', aggfunc = 'count', 
                    fill_value = 0, margins = True).reset_index()

category,lat:long,Авто,Дача,Спорт,Электроника,All
0,53.890790:27.525773,1,0,0,1,2
1,55.733670:37.587874,0,1,0,1,2
2,55.812308:27.525555,0,1,0,1,2
3,55.824108:37.550309,0,0,1,1,2
4,55.834784:37.565415,0,0,1,1,2
5,55.846231:37.532002,1,0,0,0,1
6,56.011552:36.545722,1,0,0,0,1
7,All,3,2,2,5,12


Следующим шагом - объединить точки по квадратам, и сделать группировку не по самим точкам, а по квадратам